In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [25]:
# CNN 텐서 만들기
def build_CNN_classifier(x):
    # x로 들어오는 image shape = [55000, 784]
    # 위 x를 4차원 형태로 reshape -> stride, pooling의 shape인자와 동일
    '''
    [batch, height, width, channel]
    - batch:데이터 개수(몇 개가 들어올지 모르니 유연하게 -1로 설정
    - height, width : 이미지 가로,세로 길이
    - channel: 이미지 표현하는 RGB(3개의 색깔로 모두 표현함)이기 때문에 1인자로 1차원 추가 -> 다른 숫자가 오면안되나.?질문하기..
    '''
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    # 1번째 컨볼루션 필터 W값
    '''<필터 shape설정만 독특하게 다름!>
    [height, width, 입력 channel, 출력 channel]
    - height, width: 이미지 가로, 세로 길이
    - 입력 channel: 들어오는 이미지 개수
    - 출력 channel: 들어오는 이미지 1개에 대해 필터들을 적용하니까 필터개수 만큼 출력됨!
    '''
        # truncated_normal:일반 random_normal로 하면 -무한대와 +무한대에 근사한 값들도 포함되어 설정할 수 있으므로 그 경우에 학습시간이 오래걸림
        # 따라서 -무한대, +무한대 부분을 제외한(자른=truncated) truncated_normal 사용
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 32], stddev=5e-2))
    
    # 1번째에 적용된 필터들 거친 후 필터들 개수만큼 출력된 특징들에 각각 bias 연산하도록 bias할당
    # 여기서는 상수(constant)로 고정시켰지만 random값으로 설정해도 됨
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    
    # 필터와 bias 적용 단계와 그 후 나온 특징들에 대해 활성함수 적용단계까지 한 번에!
    '''<strides shape 인자>
    [batch, height, width, channel]
    - batch: 데이터 개수(이미지 1개가 들어오니 1)
    - height, width: 가로,세로 1칸씩 띄워서 stride 적용
    - channel: 이미지 표현 RGB니깐 인자를 1로 추가해 3차원으로 설정
    '''
        #padding='SAME'이면 입력 이미지 size와 필터링 걸친 후 나오는 특징 이미지 size와 같도록 하자!-가로,세로로 2차원이니까 conv2d사용?
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
    
    # pooling 적용
    '''<pooling shape인 kszie 인자>
    [batch, height, width, channel]
    - batch: 데이터 개수(이미지 1개가 들어오니 1)
    - height, width: pooling할 size를 가로, 세로 2로 설정
    - channel: 이미지 표현 RGB니까 1을 추가해 3차원으로 설정
    @위에서 padding='SAME'으로 필터링 거친후 출력 size가 입력 size와 동일하니 [1, 28, 28, 1]이다. 
    따라서 pooling size(ksize)를 2 by 2로 설정했으니 size가 "28 -> 14"로 1/2만큼 줄어듦!
    '''
        # strides인자와 padding 인자 의미는 위와 동일
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # 2번째 컨볼루션 필터 W값
    '''
    - 필터 shape의 인자 3번째가 32인 이유는 W_conv1에서 필터개수를 32개로 설정해 필터링 거친 후 특징들도 32개이기 때문에 입력 channel=32로 설정
    - 이제 2번째 컨볼루션에서 필터 개수를 64개로 설정해서 특징을 64개만큼 도출
    '''
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 32, 64], stddev=5e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
    # h_pool1결과로 각 필터의 특징 size가 14 by 14인 상태인데, 여기서 pooling size를 2 by 2로 설정
    # 2번째 pooling을 거친 후 각 필터의 특징 size는 7 by 7임.
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Fully Connected layer1 정의
    '''<Fully Conntected Wegiht값 size설정 
    [입력channel, 출력channel]
    - shape 첫 번째 인자: 2번의 pooling 모두 거친 후의 이미지 가로와 세로,
      그리고 64개의 필터들로 추출된 특징 64개를 곱해 모든 특징값들을 한 곳으로 모으기
    - shape 두 번째 인자: 1024개 만큼의 출력 size를 임의로 설정?
    '''
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[7*7*64, 1024], stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    # 2번의 CNN을 거친 결과의 4차원 shape을 1차원으로 쫘악 펴주기- 데이터개수(행)는 가변적(-1)으로, 열(column)은 모든 특징값들 개수
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    # flatten해준 특징값들을 W_fc1과 곱한 후 summation, 그리고 bias값 더하기!
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    # 최종 분류 결과값을 도출할 Fully connected layer2 정의
    W_output = tf.Variable(tf.truncated_normal(shape=[1024, 10], stddev=5e-2))
    b_output = tf.Variable(tf.constant(0.1, shape=[10]))
    # flatten한 후 활성함수 적용한 값들에 최종 Weight, bias값 기반으로 활성함수 적용!
    logits = tf.matmul(h_fc1, W_output) + b_output
    # softmax 활성함수로 최종 10개의 클래스 중 한개로 분류
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

In [8]:
image = mnist.train.images
print(image.shape)

(55000, 784)


In [10]:
reshape = tf.reshape(image, [-1, 28, 28, 1])

In [21]:
image.reshape(-1, 28, 28, 1)[0].shape

(28, 28, 1)